In [0]:
import tensorflow as tf 
import pandas as pd
import numpy as np 

from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle

In [0]:
# Download the dataset from GitHub repo

!curl -O https://media.githubusercontent.com/media/pietromoretto/lanGuesser/master/dataset.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  109M  100  109M    0     0  40.4M      0  0:00:02  0:00:02 --:--:-- 40.3M


In [0]:
!ls

dataset.csv  keras_saved_model.h5  __pycache__	tokenizer_state.pkl
gdrive	     predictor.py	   sample_data


In [0]:
convert = {
    ".java": "JAVA",
    ".py": "PYTHON",
    ".html": "HTML",
    ".css": "CSS",
    ".php": "PHP",
    ".sql": "SQL",
    ".js": "JAVASCRIPT",
    ".go": "GO",
}

In [0]:
# Create the dataframe

data = pd.read_csv("dataset.csv")

data = shuffle(data, random_state=22)
data = data.dropna()

data.head()

,content,language
8870,/*\n * Copyright 2013-2016 the original author...,JAVA
9607,"/*\n * Copyright (c) 2011 LinkedIn, Inc\n * \n...",JAVA
8424,package org.knowm.xchange.btce.v3.service.poll...,JAVA
4995,"package qb\n\nimport (\n\t""fmt""\n\t""strings""\n...",GO
219,"﻿/*\nCopyright (c) 2003-2013, CKSource - Frede...",JAVASCRIPT


In [0]:
# Print how many languages are in the dataset

for k, v in convert.items():
    cust = data["language"].apply(lambda lan: lan == v)
    print(v, len(cust[cust == True]))

C 0
JAVA 1931
PYTHON 1630
RUBY 0
HTML 574
CSS 715
PHP 2033
SQL 1229
C# 0
JAVASCRIPT 435
GO 1487


In [0]:
data.describe()

,content,language
count,10034,10034
unique,10034,8
top,// Copyright 2011 The Go Authors. All rights r...,PHP
freq,1,2033


In [0]:
# One Hot Encoding on programming languages (labels)

encoder = LabelBinarizer()
lang_encoded = encoder.fit_transform(data["language"].values)
num_lang = len(lang_encoded[0])

print(data['language'].values[0])
print(encoder.classes_)
print(lang_encoded[0])

JAVA
['CSS' 'GO' 'HTML' 'JAVA' 'JAVASCRIPT' 'PHP' 'PYTHON' 'SQL']
[0 0 0 1 0 0 0 0]


In [0]:
# Split data into train and test sets (80 / 20)

train_size = int(len(data) * .8)
test_size = (len(data) - train_size)

print(f"Train size: {train_size}")
print (f"Test size: {test_size}")

Train size: 8027
Test size: 2007


In [0]:
# Split labels into train and test sets

train_lang = lang_encoded[:train_size]
test_lang = lang_encoded[train_size:]

In [0]:
# Tokenize the features

from tensorflow.keras.preprocessing import text

VOCAB_SIZE = 4000 

train_content = data['content'].values[:train_size]
test_content = data['content'].values[train_size:]

tokenizer = text.Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(train_content)

body_train = tokenizer.texts_to_matrix(train_content)
body_test = tokenizer.texts_to_matrix(test_content)

In [0]:
# Preview the first input from our training data

print(len(body_train[0]))
print(body_train[0])

4000
[0. 1. 1. ... 0. 0. 0.]


In [0]:
# Create the model

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(50, input_shape=(VOCAB_SIZE,), activation='relu'))
model.add(tf.keras.layers.Dense(25, activation='relu'))
model.add(tf.keras.layers.Dense(num_lang, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 50)                200050    
_________________________________________________________________
dense_14 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_15 (Dense)             (None, 8)                 208       
Total params: 201,533
Trainable params: 201,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Train and evaluate the model

model.fit(body_train, train_lang, epochs=10, batch_size=128, validation_split=0.1)

print("Eval loss/accuracy: {}".format(model.evaluate(body_test, test_lang, batch_size=128)))

Train on 7224 samples, validate on 803 samples
Epoch 1/10
7224/7224 [==============================] - 1s 189us/sample - loss: 0.2140 - acc: 0.9210 - val_loss: 0.1040 - val_acc: 0.9622
Epoch 2/10
7224/7224 [==============================] - 1s 92us/sample - loss: 0.0532 - acc: 0.9823 - val_loss: 0.0473 - val_acc: 0.9868
Epoch 3/10
7224/7224 [==============================] - 1s 91us/sample - loss: 0.0228 - acc: 0.9941 - val_loss: 0.0366 - val_acc: 0.9911
Epoch 4/10
7224/7224 [==============================] - 1s 92us/sample - loss: 0.0132 - acc: 0.9969 - val_loss: 0.0278 - val_acc: 0.9939
Epoch 5/10
7224/7224 [==============================] - 1s 89us/sample - loss: 0.0085 - acc: 0.9981 - val_loss: 0.0262 - val_acc: 0.9947
Epoch 6/10
7224/7224 [==============================] - 1s 89us/sample - loss: 0.0061 - acc: 0.9988 - val_loss: 0.0252 - val_acc: 0.9946
Epoch 7/10
7224/7224 [==============================] - 1s 87us/sample - loss: 0.0046 - acc: 0.9992 - val_loss: 0.0256 - val_acc: 

In [0]:
# Export the model to a file

model.save('keras_saved_model.h5')

In [0]:
# Save the processor state of the tokenizer

import pickle

with open('./tokenizer_state.pkl', 'wb') as f:
  pickle.dump(tokenizer, f)

In [0]:
%%writefile predictor.py
import pickle
import os
import numpy as np


class Predictor:
  def __init__(self, model, processor):
    self._model = model
    self._processor = processor
  
  def predict(self, instances, **kwargs):
    preprocessed_data = self._processor.texts_to_matrix(instances)
    predictions = self._model.predict(preprocessed_data)
    return predictions.tolist()

  @classmethod
  def from_path(cls, model_dir):
    import tensorflow.keras as keras
    model = keras.models.load_model(
      os.path.join(model_dir,'keras_saved_model.h5'))
    with open(os.path.join(model_dir, 'tokenizer_state.pkl'), 'rb') as f:
      processor = pickle.load(f)

    return cls(model, processor)

Overwriting predictor.py


In [0]:
# Test model on custom data

from predictor import Predictor


test = [
    """var rows = prompt("How many rows for your multiplication table?");
    var cols = prompt("How many columns for your multiplication table?");
    if(rows == "" || rows == null)
   		 rows = 10;
    if(cols== "" || cols== null)
   		 cols = 10;
    createTable(rows, cols);
    function createTable(rows, cols)
    {
      var j=1;
      var output = "<table border='1' width='500' cellspacing='0'cellpadding='5'>";
      for(i=1;i<=rows;i++)
      {
    	output = output + "<tr>";
        while(j<=cols)
        {
  		  output = output + "<td>" + i*j + "</td>";
   		  j = j+1;
   		}
   		 output = output + "</tr>";
   		 j = 1;
    }
    output = output + "</table>";
    document.write(output);
    }""",
    """<!DOCTYPE html>
<html>
<body>

<h2>HTML Links</h2>
<p>HTML links are defined with the a tag:</p>

<a href="https://www.w3schools.com">This is a link</a>

</body>
</html>""",
    """a:link {
  color: gray;
}

a:visited {
  color: green;
}

a:hover {
  color: rebeccapurple;
}

a:active {
  color: teal;
}""",
    """import pickle
import os
import numpy as np


class Predictor:
    def __init__(self, model, processor):
        self._model = model
        self._processor = processor
        self.languages = [
            "C",
            "C#",
            "CSS",
            "GO",
            "HTML",
            "JAVA",
            "JAVASCRIPT",
            "PHP",
            "PYTHON",
            "RUBY",
            "SQL",
        ]

    def predict(self, instances, **kwargs):
        preprocessed_data = self._processor.texts_to_matrix(instances)
        predictions = self._model.predict(preprocessed_data)
        return predictions.tolist()

    @classmethod
    def from_path(cls, model_dir):
        import tensorflow.keras as keras

        model = keras.models.load_model(os.path.join(model_dir, "keras_saved_model.h5"))
        with open(os.path.join(model_dir, "tokenizer_state.pkl"), "rb") as f:
            processor = pickle.load(f)

        return cls(model, processor)

""",
    """// In Go, an _array_ is a numbered sequence of elements of a
// specific length.

package main

import "fmt"

func main() {

    // Here we create an array `a` that will hold exactly
    // 5 `int`s. The type of elements and length are both
    // part of the array's type. By default an array is
    // zero-valued, which for `int`s means `0`s.
    var a [5]int
    fmt.Println("emp:", a)

    // We can set a value at an index using the
    // `array[index] = value` syntax, and get a value with
    // `array[index]`.
    a[4] = 100
    fmt.Println("set:", a)
    fmt.Println("get:", a[4])""",
    """
CREATE TABLE emp (
empno INT PRIMARY KEY,
ename VARCHAR(10),
job VARCHAR(9),
mgr INT NULL,
hiredate DATETIME,
sal NUMERIC(7,2),
comm NUMERIC(7,2) NULL,
dept INT)
begin
insert into emp values
    (1,'JOHNSON','ADMIN',6,'12-17-1990',18000,NULL,4)
insert into emp values
    (2,'HARDING','MANAGER',9,'02-02-1998',52000,300,3)
insert into emp values
    (3,'TAFT','SALES I',2,'01-02-1996',25000,500,3)
insert into emp values
    (4,'HOOVER','SALES I',2,'04-02-1990',27000,NULL,3)
insert into emp values
    (5,'LINCOLN','TECH',6,'06-23-1994',22500,1400,4)
insert into emp values
    (6,'GARFIELD','MANAGER',9,'05-01-1993',54000,NULL,4)
insert into emp values
    (7,'POLK','TECH',6,'09-22-1997',25000,NULL,4)
insert into emp values
    (8,'GRANT','ENGINEER',10,'03-30-1997',32000,NULL,2)
insert into emp values
    (9,'JACKSON','CEO',NULL,'01-01-1990',75000,NULL,4)
insert into emp values
    (10,'FILLMORE','MANAGER',9,'08-09-1994',56000,NULL,2)
insert into emp values
    (11,'ADAMS','ENGINEER',10,'03-15-1996',34000,NULL,2)
insert into emp values
    (12,'WASHINGTON','ADMIN',6,'04-16-1998',18000,NULL,4)
insert into emp values
    (13,'MONROE','ENGINEER',10,'12-03-2000',30000,NULL,2)
insert into emp values
    (14,'ROOSEVELT','CPA',9,'10-12-1995',35000,NULL,1)
end
CREATE TABLE dept (
deptno INT NOT NULL,
dname VARCHAR(14),
loc VARCHAR(13))
begin
insert into dept values (1,'ACCOUNTING','ST LOUIS')
insert into dept values (2,'RESEARCH','NEW YORK')
insert into dept values (3,'SALES','ATLANTA')
insert into dept values (4, 'OPERATIONS','SEATTLE')
end
""",
    """if ( isset( $_FILES['fupload'] ) ) {

     print "name: ".     $_FILES['fupload']['name']       ."<br />";
     print "size: ".     $_FILES['fupload']['size'] ." bytes<br />";
     print "temp name: ".$_FILES['fupload']['tmp_name']   ."<br />";
     print "type: ".     $_FILES['fupload']['type']       ."<br />";
     print "error: ".    $_FILES['fupload']['error']      ."<br />";

     if ( $_FILES['fupload']['type'] == "image/gif" ) {

         $source = $_FILES['fupload']['tmp_name'];
         $target = "upload/".$_FILES['fupload']['name'];
         move_uploaded_file( $source, $target );// or die ("Couldn't copy");
         $size = getImageSize( $target );

         $imgstr = "<p><img width=\"$size[0]\" height=\"$size[1]\" ";
         $imgstr .= "src=\"$target\" alt=\"uploaded image\" /></p>";

         print $imgstr;
     }
 }""",
    """import java.util.Scanner;
public class Demo {

    public static void main(String[] args) {

    	int year;
    	Scanner scan = new Scanner(System.in);
    	System.out.println("Enter any Year:");
    	year = scan.nextInt();
    	scan.close();
        boolean isLeap = false;

        if(year % 4 == 0)
        {
            if( year % 100 == 0)
            {
                if ( year % 400 == 0)
                    isLeap = true;
                else
                    isLeap = false;
            }
            else
                isLeap = true;
        }
        else {
            isLeap = false;
        }

        if(isLeap==true)
            System.out.println(year + " is a Leap Year.");
        else
            System.out.println(year + " is not a Leap Year.");
    }
}""","""print("A")""",
    """function a(){
          console.log(3);
        }""",
    """select * from table"""
]


predictor = Predictor.from_path(".")
results = predictor.predict(test)

for result in results:
    print("Predicted labels:")
    max_pred = -10000
    max_i = -1
    for i, pred in enumerate(result):
        if pred > max_pred:
            max_i = i
            max_pred = pred

    print(encoder.classes_[max_i], max_pred)
    print()


Predicted labels:
JAVASCRIPT 0.7671231031417847

Predicted labels:
HTML 0.9057226181030273

Predicted labels:
CSS 0.971668541431427

Predicted labels:
PYTHON 0.9999939203262329

Predicted labels:
GO 0.9993784427642822

Predicted labels:
SQL 0.9999995231628418

Predicted labels:
PHP 0.8473194241523743

Predicted labels:
JAVA 0.999915599822998

Predicted labels:
PYTHON 0.5491448640823364

Predicted labels:
JAVASCRIPT 0.4530501663684845

Predicted labels:
SQL 0.9729054570198059

